In [1]:
import pandas as pd 
import os
import numpy as np
from tqdm import tqdm
os.chdir("./..")
import sys
sys.path.append('./src/sena_discrepancy_vae')
from utils import Norman2019DataLoader, Wessel2023HEK293DataLoader
import json

In [2]:
def get_datahandler(dataset_name, batch_size=32):

    if 'Norman2019' in dataset_name:
        data_handler = Norman2019DataLoader(batch_size=batch_size)
    elif dataset_name == 'wessel_hefk293':
        data_handler = Wessel2023HEK293DataLoader(batch_size=batch_size)

    return data_handler

In [3]:
def generate_differential_activation_scores(df, data_handler, layer='fc1'):

    #get activity score df of specific layer
    activity_score = df[layer]

    #define control cells
    ctrl_cells = activity_score[activity_score.index == 'ctrl'].to_numpy()

    #get perturbation targets
    ptb_targets = [x for x in set(activity_score.index) if x != 'ctrl']

    #get gene go dict
    gene_go_dict = data_handler.gene_go_dict
    genename_ensemble_dict = data_handler.ensembl_genename_mapping_rev

    #initialize analysis dataframe
    analysis_df_l = []

    for knockout in tqdm(ptb_targets):

        gos = activity_score.columns
        for i, geneset in enumerate(gos):

            #some knockouts may not have any targeted gene set
            if knockout not in genename_ensemble_dict:
                continue

            #get knockout cells
            knockout_cells = activity_score[activity_score.index == knockout].to_numpy()

            #compute affected genesets
            belonging_genesets = [geneset for geneset in gos if geneset in gene_go_dict[genename_ensemble_dict[knockout]]]

            #compute differential activation score
            score = abs(ctrl_cells[:,i].mean() - knockout_cells[:,i].mean())

            #append
            analysis_df_l.append([knockout, geneset, score, geneset in belonging_genesets])


    #build the dataframe
    analysis_df = pd.DataFrame(analysis_df_l)
    analysis_df.columns = ['knockout', 'geneset', 'score', 'affected']

    return analysis_df

In [4]:
#define folder path
folder_path = os.path.join('results', 'example')

#load the activation scores
activation_scores_dict = pd.read_pickle(os.path.join(folder_path, 'activation_scores.pickle'))

#get datasetname
with open(os.path.join(folder_path,'config.json'), 'r') as file:
        config_file = json.load(file)
dataset_name = config_file["dataset_name"]

#load data handler
data_handler = get_datahandler(dataset_name)

#we can generate the differential scores of any layer by just calling the previously defined function
activation_scores_fc1 = generate_differential_activation_scores(activation_scores_dict, data_handler, layer='fc1')

100%|██████████| 3/3 [00:00<00:00,  5.50it/s]


In [5]:
activation_scores_fc1

,knockout,geneset,score,affected
0,CD55,GO:0000076,0.000494,False
1,CD55,GO:0000212,0.000096,False
2,CD55,GO:0000289,0.000044,False
3,CD55,GO:0000727,0.000430,False
4,CD55,GO:0001516,0.000355,False
...,...,...,...,...
673,CD46,GO:2000810,0.000156,False
674,CD46,GO:2001014,0.000336,False
675,CD46,GO:2001032,0.000337,False
676,CD46,GO:2001044,0.000240,False
